# chat UI in fastHTML

> chat UI implemented in fastHTML

In [ ]:
#| default_exp chat_ui

In [ ]:
#| hide
from nbdev.showdoc import *

## Chat App initialization

Start by creating the chat application with `FastHTML`.

In [ ]:
#| export
import uvicorn
import uuid
import os
import asyncio
from fasthtml.common import *
from fastcore.parallel import startthread
from typing import Callable, Optional

from llmcam.fn_to_fc import capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file
from llmcam.fn_to_fc import tool_schema, complete, form_msg
from llmcam.store import add_api_tools, add_function_tools, remove_tools
from llmcam.store import execute_handler_core, handler_schema
from llmcam.yolo import detect_objects
from llmcam.dtcam import cap
from llmcam.file_manager import list_image_files, list_detection_files
from llmcam.plotting import plot_object
from llmcam.notification import notification_stream_core, process_notification_schema, StreamThread
from llmcam.bash_command import *

### Pseudo-database and session FC setup

Our current tool stack include several functions that capitalize on having one or more universal pointers (store manager functions depend on a global `tools`, notifications depend on a global `stream_thread` and sender function). However, such universal pointers will not work in web settings, where multiple users can simultaneously access the website. Therefore, the idea is to introduce sessions and universal pointers that are mappings of these sessions to actual values the tools can use. We also introduce `session_id` as a new metadata field for our tools and implement custom fixup functions that extract the correct values from the mappings and pass down to our already implemented tools.

In [ ]:
#| export
# Set up database for information per session
session_messages = {}  # Messages for each session
session_tools = {}  # Tools for each session
session_notis = {}  # Sender and notification streams for each session

In [ ]:
#| export
# Set up default tools
default_tools = [tool_schema(fn) for fn in (
    capture_youtube_live_frame_and_save, 
    ask_gpt4v_about_image_file,
    detect_objects,
    cap,
    list_image_files,
    list_detection_files,
    plot_object,
    execute_bash_command,
)]

Here, we implement the schema preparation to attach `session_id` as metadata and `fixup` function for store manager functions and actual `fixup` function that extracts correct `tools` pointer to pass to `execute_handler_core`. 

In [ ]:
#| export
# Utility functions to manage tools per session
def prepare_handler_schemas(
    session_id: str,  # Session ID to use
    fixup: Callable = None,  # Optional function to fix up the execution
):
    return [
        handler_schema(function, service_name="toolbox_manager", fixup=fixup, session_id=session_id) for \
        function in [add_api_tools, add_function_tools, remove_tools]
    ]

def execute_handler(
    function_name: str,  # Name of the function to execute
    session_id: str,  # Session ID to use
    **kwargs,  # Additional arguments to pass to the function
):
    tools = session_tools[session_id]
    return execute_handler_core(tools, function_name, **kwargs)

The notifications system is quite more complicated. It relies on also subtools to pass in sub GPT in the separate thread running notification stream. These subtools include a tool to send notification and a tool to stop the stream. The default utilities function for this uses a common global pointer `stream_thread`, which will not work in this case. Hence, we need to implement the new utilities function:

- `execute_send_notification`: Instead of implementing the actual function, a `fixup` is implemented to capitalize on the metadata field `session_id`. This function also utilizes the feature that a websocket sender function can be saved in global collections -  [example with real time Chat App](https://docs.fastht.ml/explains/websockets.html#real-time-chat-app). Therefore, it simply retrieves the sender function with `session_id` and uses it to send the message.  
- `execute_stopper`: Instead of implementing the actual function, a `fixup` is implemented to capitalize on the metadata field `session_id` and `noti_id`. The idea is that each notification stream is given a unique ID and saved in a mapping of IDs to notification streams. This mapping is also retrievable by `session_id`.

We also need to implement a custom `start_notification_stream` function which will be used as a tool by GPT Function Calling. This function will utilize `session_id` and create a unique `noti_id` to define schemas for subtools with these values as metadata. It also defines these schemas such that the `module` metadata is missing to ensure the attached `fixup` function will be called instead. However, because it uses a metadata field `session_id`, it will also need to have a `fixup` function.

In [ ]:
#| export
# Utility functions to manage notifications per session
def execute_send_notification(function_name, session_id, msg, **kwargs):
    """Fixup function to send a notification."""
    global session_notis
    sender, _ = session_notis[session_id]  # Get the sender
    sender(msg)
    return 'Notification sent'

def execute_stopper(function_name, session_id, noti_id, **kwargs):
    """Fixup function to stop a notification stream."""
    global session_notis
    _, notis = session_notis[session_id]  # Get the notification streams 
    notis[noti_id].stop()  # Stop the stream with the given ID
    return 'Notification stream stopped'

def start_notification_stream(
    session_id: str,  # Session ID to use
    messages: list,  # All the previous messages in the conversation
):
    """Start a notification stream to monitor a process described in messages."""
    global session_notis
    global session_tools

    _, notis = session_notis[session_id]  # Get the notification streams

    # Define a new notification stream with a unique ID
    noti_id = str(uuid.uuid4())
    
    # Describe the sender and stopper functions
    sender_schema = {
        'type': 'function',
        'function': {
            'name': 'send_notification',
            'description': 'Send a notification with a message',
            'metadata': {
                'session_id': session_id
            },
            'parameters': {'type': 'object',
                'properties': {'msg': {'type': 'string',
                'description': 'No description provided.'}},
                'required': ['msg']},
            'fixup': f"{execute_send_notification.__module__}.{execute_send_notification.__name__}"
        },
    }

    stopper_schema = {
        'type': 'function',
        'function': {
            'name': 'stop_notification',
            'description': 'Stop the notification stream',
            'parameters': {'type': 'object', 'properties': {}, 'required': []},
            'metadata': {
                'session_id': session_id,
                'noti_id': noti_id
            },
            'fixup': f"{execute_stopper.__module__}.{execute_stopper.__name__}"
        }
    }

    # Define a function to start the stream
    def stream_starter(tools, messages):
        notis[noti_id] = StreamThread(noti_id, tools, messages)
        notis[noti_id].start()

    # Extract the tools for the session
    tools = session_tools[session_id]

    # Start the notification stream
    notification_stream_core(
        tools, 
        messages,
        stream_starter=stream_starter,
        send_notification_schema=sender_schema,
        stream_stopper_schema=stopper_schema
    )

    # Return the ID of the notification stream  
    return f"Notification stream started with ID {noti_id}" 

Define the `fixup` function for starting a notification stream which passes in `session_id` to `start_notification_stream`. Define also a function to attach `session_id` as metadata for the notification FC schema and attach the `fixup` function.

In [ ]:
#| export
def prepare_notification_schemas(
        session_id: str,  # Session ID to use
        fixup: Callable = None,  # Optional function to fix up the execution
    ):  # Prepare the notification schema
    schema = process_notification_schema(start_notification_stream)  # Get the schema for starting notification stream
    # Set additional metadata
    schema['function']['metadata']['session_id'] = session_id  
    if fixup: schema['function']['fixup'] = f"{fixup.__module__}.{fixup.__name__}"
    return schema

def execute_start_notification_stream(function_name, session_id, messages, **kwargs):
    """Fixup function to start a notification stream."""
    return start_notification_stream(session_id, messages)

In [ ]:
#| export
def init_session(session_id: Optional[str] = None):
    if session_id is None or session_id not in session_messages:
        # Initialize tools in session tools and create a session ID
        session_id = str(uuid.uuid4())

        # Add default tools, prepare handler schemas, and prepare notification schema
        session_tools[session_id] = []
        session_tools[session_id].extend(default_tools)
        session_tools[session_id].extend(prepare_handler_schemas(session_id, execute_handler))
        session_tools[session_id].append(prepare_notification_schemas(session_id, execute_start_notification_stream))

        # Initialize messages in session messages
        session_messages[session_id] = []
    
    return session_id

### Application

In [ ]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink,
        Link(rel="icon", href=f"""{os.getenv("LLMCAM_DATA", "../data").split("/")[-1]}/favicon.ico""", type="image/png"),
        Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"),
        Script(src="https://unpkg.com/htmx.org"),
        Style("p {color: black;}"),
        Style("li {color: black;}"),
        MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']))
app = FastHTML(hdrs=hdrs, exts="ws")

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [ ]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(
        msg: str,  # Message to display
        user: bool  # Whether the message is from the user or assistant
    ):  # Returns a Div containing the chat bubble
    # Set class to change displayed style of bubble
    content_class = "chat-bubble chat-bubble-primary" if user else ""
    content_class += " marked py-2"
    return  Div(cls=f"chat chat-end py-4" if user else "py-4")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    msg,
                    cls=content_class,
                )
            )

In [ ]:
show_doc(ChatMessage)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L183){target="_blank" style="float:right; font-size:smaller"}

### ChatMessage

>      ChatMessage (msg:str, user:bool)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | str | Message to display |
| user | bool | Whether the message is from the user or assistant |

For the chat input, set the name for submitting a new message via form.

In [ ]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():  # Returns an input field for the user message
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

In [ ]:
show_doc(ChatInput)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L201){target="_blank" style="float:right; font-size:smaller"}

### ChatInput

>      ChatInput ()

### Action Buttons

Simple actions for creating a new message from the user side.

In [ ]:
#| export
def ActionButton(
        session_id: str,  # Session ID to use
        content: str,  # Text to display on the button
        message: str = None  # Message to send when the button is clicked
    ):  # Returns a button with the given content

    return Form(
        ws_send=True,
        hx_ext='ws', ws_connect='/wschat',
    )(
        Hidden(session_id, name="session_id"),
        Hidden(content if message is None else message, name="msg"),
        Button(
            content, 
            cls="btn btn-secondary rounded-2 h-fit", 
        )
    )

def ActionPanel(
        session_id: str  # Session ID to use
    ):  # Returns a panel of action buttons
    return Div(
        P("Quick actions", cls="text-lg text-black"),
        ActionButton(session_id, "Introduce your model GPT-4o"),
        ActionButton(session_id,
            "Extract information from a YouTube Live", 
            "Capture and extract information from a YouTube Live. Use the default link."),
        cls="flex flex-col h-fit gap-4 py-4 px-4"
    )

In [ ]:
show_doc(ActionButton)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L208){target="_blank" style="float:right; font-size:smaller"}

### ActionButton

>      ActionButton (session_id:str, content:str, message:str=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| session_id | str |  | Session ID to use |
| content | str |  | Text to display on the button |
| message | str | None | Message to send when the button is clicked |

In [ ]:
show_doc(ActionPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L226){target="_blank" style="float:right; font-size:smaller"}

### ActionPanel

>      ActionPanel (session_id:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| session_id | str | Session ID to use |

### Tools panel

Sidebar-panel for displaying current list of available (loaded) tools in a user-session.

In [ ]:
#| export
def ToolPanel(
        session_id: str  # Session ID to use
    ):  # Returns a panel of usable tools

    available_services = session_tools.get(session_id, [])

    # Generate list items for each available tool
    items = []
    if available_services:
        for service in available_services:
            service_desc = service['function']['description']
            items.append(Li(f"{service_desc}", cls="text-sm text-black"))
    else:
        items.append(Li("No services available", cls="text-sm italic text-gray-500"))

    return Div(
        P("Available Tools", cls="text-lg text-black"),
        Ul(*items, cls="list-disc list-inside px-6", style="max-height: 800px; overflow-y:auto;"),
        id="toollist",
        cls="flex flex-col h-fit gap-4 py-4 px-4"
    )

In [ ]:
show_doc(ToolPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L239){target="_blank" style="float:right; font-size:smaller"}

### ToolPanel

>      ToolPanel (session_id:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| session_id | str | Session ID to use |

### Notifications

The idea of sending notifications from a background task / websocket with FastHTML is to send an HTMX update, then detect and extract information from the event via a document event listener.

In [ ]:
#| export
def NotiMessage(
        message: str = "No message"  # Message to display
    ):  # Returns a notification message hidden from the UI view
    return Hidden(message, id="notification", cls="text-black")

def NotiButton(
        session_id: str  # Session ID to use
    ):  # Returns a hidden button to trigger notification websocket connection
    return Form(
        ws_send=True,
        hx_ext='ws', ws_connect='/wsnoti',
        style="display: none;"
    )(
        Hidden(session_id, name="session_id"),
        Button(
            "Notification", 
            id="connect-btn", 
            cls="btn btn-primary rounded-2 h-fit", 
            style="display: none;"
        )
    )

In [ ]:
#| export
# Event listener to handle notifications when the element #notification is loaded
noti_script = Script("""
    // Automatically click the hidden button to connect to the notification websocket
    window.addEventListener('load', function() {
        let connectButton = document.querySelector('#connect-btn');
        if (connectButton) {
            connectButton.click();
            console.log("Hidden button clicked on page load!");
        }
    });
                     
    // Listen for the htmx:load event on the document body
    document.body.addEventListener('htmx:load', function(event) {
        if (event.target.id === "notification") {
            let htDivElement = event.detail.elt; // Extract the HtDiv element

            // Find the input element inside the HtDiv and extract its value
            let inputElement = htDivElement.querySelector('input');
            if (inputElement) {
                let inputValue = inputElement.value;
                alert(inputValue);
            } else {
                console.log("Input element not found.");
            }
        }
    });
""")

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [ ]:
#| export
scroll_script = Script("""
  // Function to scroll to the bottom of an element
  function scrollToBottom(element) {
    element.scrollTop = element.scrollHeight;
  }

  // Reference the expanding element
  const expandingElement = document.getElementById('chatlist');

  // Observe changes to the element's content and scroll down automatically
  const observer = new MutationObserver(() => {
    scrollToBottom(expandingElement);
  });

  // Start observing the expanding element for changes
  observer.observe(expandingElement, { childList: true, subtree: true });
""")

In [ ]:
#| export
title_script = Script("""
    // Function to set the title of the page
    document.title = "LLMCAM";
""")

In [ ]:
#| export
@app.get('/')
async def index(session):
    # Initialize the session
    session_id = init_session(session_id=session.get('session_id'))
    
    # Set up the chat UI
    sidebar = Div(
        ActionPanel(session_id=session_id),
        ToolPanel(session_id=session_id),
        NotiButton(session_id=session_id),
        NotiMessage(),
        cls="w-[50vw] flex flex-col p-0",
        style="background-color: WhiteSmoke;"
    )
    page =  Div(cls="w-full flex flex-col p-0")(  # Main page
        Form(
            ws_send=True,
            hx_ext='ws', ws_connect='/wschat',
            cls="w-full flex flex-col px-24 h-[100vh]"
        )(
            Hidden(session_id, name="session_id"),
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10 p-4")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2 p-4")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl"))
            ),
            scroll_script
        ),
    )
    return Main(
        noti_script,
        title_script,
        sidebar,
        page, 
        title="Chatbot",
        data_theme="wireframe",
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

### Notification websockets

In [ ]:
#| export
def noti_disconnect(ws):
    """Remove session ID from session notification sender on websocket disconnect"""
    session_id = ws.scope.get("session_id")
    if session_id in session_messages:
        del session_messages[session_id]
    if session_id in session_tools:
        del session_tools[session_id]
    if session_id in session_notis:
        del session_notis[session_id]

In [ ]:
#| export
@app.ws('/wsnoti')
async def wsnoti(ws, send, session_id: str):
    # Initialize the session
    session_id = init_session(session_id=session_id)

    # Set the session ID in the websocket scope
    ws.scope["session_id"] = session_id

    # Set up the notification sender for the session
    def send_noti(message):
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:  # No current event loop in this thread
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
    
        if loop.is_running():
            # Schedule the task on the running loop
            asyncio.create_task(send(Div(NotiMessage(message), id="notification", hx_swap_oob="true")))
        else:
            # Create and run a new loop
            loop.run_until_complete(send(Div(NotiMessage(message), id="notification", hx_swap_oob="true")))
    
    session_notis[session_id] = (send_noti, {})

    # Send a notification to the client
    send_noti("Notification service enabled.")

Test usage:

In [ ]:
#| eval: false
first_noti_sender = list(session_notis.values())[0][0]
first_noti_sender("Test message.")

### Chat websockets

When connecting to websockets for chat, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [ ]:
#| export
# On websocket disconnect, remove the session ID from the session messages and tools
def chat_disconnect(ws):
    """Remove session ID from session messages and tools on websocket disconnect"""
    session_id = ws.scope.get("session_id")
    if session_id in session_messages:
        del session_messages[session_id]
    if session_id in session_tools:
        del session_tools[session_id]

In [ ]:
#| export
# The chatbot websocket handler
@app.ws('/wschat', disconn=chat_disconnect)
async def wschat(ws, msg: str, send, session_id: str):
    # Initialize the session
    session_id = init_session(session_id=session_id)

    # Set the session ID in the websocket scope
    ws.scope["session_id"] = session_id

    # Set up the global variables
    global session_tools
    global execute_handler
    
    # Create chat messages from the provided contents and roles
    messages = session_messages.get(session_id, [])
    if len(messages) == 0:
        messages.append(
            form_msg(
                "system", 
"You are a helpful assistant. Use the supplied tools to assist the user. \
If asked to show or display an image or plot, do it by embedding its path starting with \
`../data/<filename>` in Markdown syntax. \
When asked to monitor or notify about a process, do not operate the condition but instead\
start a notification stream to monitor the process and \
use the available tools to stop stream or send notifications. \
By default, stop stream after one notification sent."))
    messages.append(form_msg("user", msg))
    await send(
        Div(ChatMessage(
            messages[-1]["content"],
            messages[-1]["role"] == "user"), 
        hx_swap_oob='beforeend', id="chatlist"))
    
    await send(ChatInput())  # Clear the input field
    
    # Add the user's message to the chat history
    complete(messages, session_tools[session_id])
    await send(Div(ChatMessage(
            messages[-1]["content"],
            messages[-1]["role"] == "user"), hx_swap_oob='beforeend', id="chatlist"))
    
    await send(Div(ToolPanel(session_id=session_id), hx_swap_oob='true', id='toollist'))
    return

### Static files

In case the user needs to display images, serves files from directory `../data`.

In [ ]:
#| export
# Serve files from the 'data' directory
@app.get("/data/{file_name:path}")
async def get_file(file_name: str):
    """Serve files dynamically from the 'data' directory."""
    data_path = os.getenv("LLMCAM_DATA", "../data")
    file_path = Path(data_path) / file_name
    if file_path.exists():
        return FileResponse(file_path)
    return {"error": f"File '{file_name}' not found"}

## Runner

In addition to the main app, an utility function is implemented to run the app just by importing and executing this function to a Python file.

In [ ]:
#| export
import asyncio
import time

def llmcam_chatbot(
        host="0.0.0.0",  # The host to listen on
        port=5001,  # The port to listen on
    ):
    # Import app from chat_ui base module
    from llmcam.chat_ui import app

    # Initialize session tools and execute handler
    session_tools = {}
    globals()["session_tools"] = session_tools

    def execute_handler(
        function_name: str,  # Name of the function to execute
        session_id: str,  # Session ID to use
        **kwargs,  # Additional arguments to pass to the function
    ):
        tools = session_tools[session_id]
        return execute_handler_core(tools, function_name, **kwargs)
    
    globals()["execute_handler"] = execute_handler
    
    server = uvicorn.Server(uvicorn.Config(app, host=host, port=port))
    async def async_run_server(server): await server.serve()
    asyncio.run(async_run_server(server))

In [ ]:
show_doc(llmcam_chatbot)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L300){target="_blank" style="float:right; font-size:smaller"}

### llmcam_chatbot

>      llmcam_chatbot (host='0.0.0.0', port=5001)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| host | str | 0.0.0.0 | The host to listen on |
| port | int | 5001 | The port to listen on |

For running while testing with Jupyter notebook, use the `JupyUvi` in `fasthtml` to run in separate thread.

In [ ]:
#| eval: false
from fasthtml.jupyter import *

server = JupyUvi(app=app)

[youtube] Extracting URL: https://www.youtube.com/watch?v=mwN6l3O1MNI
[youtube] mwN6l3O1MNI: Downloading webpage
[youtube] mwN6l3O1MNI: Downloading ios player API JSON
[youtube] mwN6l3O1MNI: Downloading mweb player API JSON
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] mwN6l3O1MNI: Downloading m3u8 information

image 1/1 /home/nghivo/tinyMLaaS/llmcam/../data/cap_2024.12.03_00:49:38_None.jpg: 384x640 9 cars, 1 traffic light, 64.5ms
Speed: 3.0ms preprocess, 64.5ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
#| eval: false
server.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()